#### Импорт необходимых библиотек

#### Объявление классов

##### Объявление функций

##### Подготовка настроек модели, создание модели

In [81]:
# Пример классификатора и токенизатора
input_dim = 2  # Зависит от метода векторизации текста
hidden_dim = 55
output_dim = 22  # Количество классов
classifier = TextClassifier(input_dim, hidden_dim, output_dim)

In [82]:
model = Model(MODEL_PATH)

#### Подготовка данных для обучения

In [83]:
train_data_dir = '../data/train/annotation'
with open(f'{train_data_dir}/hr_bot_synt.json', 'r', encoding='utf-8') as f:
    training_annotations = json.load(f)


def load_data_annotations():
    audio_files = []
    texts = []
    labels = []


    for annotation in training_annotations:
        audio_filepath = annotation['audio_filepath']
        file_id = annotation['id']
        text = annotation['text']
        label = annotation['label']

        # Добавление данных в соответствующие списки
        audio_files.append({"file": audio_filepath, "id": file_id})
        texts.append(text)
        labels.append(label)

    return audio_files, texts, labels

audio_files, texts, labels = load_data_annotations()

# Разделение данных на тренировочные и тестовые
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

tokenizer = TfidfVectorizer(max_features=input_dim)

# Подготовка текстовых данных для классификации
train_vectors = tokenizer.fit_transform(train_texts).toarray()
test_vectors = tokenizer.transform(test_texts).toarray()

train_dataset = TextDataset(train_vectors, train_labels)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

#### Функция обучения модели (+ расчет метрик каждые 5 эпох)

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001, weight_decay=0.01)

val_reference_texts = []
val_audio_files = []

def train():
    for epoch in range(30):
        for texts_batch, labels_batch in train_loader:
            texts_batch = texts_batch.float()
            optimizer.zero_grad()
            outputs = classifier(texts_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()

        # Оценка на валидационном наборе после каждой эпохи
        if (epoch + 1) % 5 == 0:  # Оценка каждые 5 эпох
            # val_hypothesis_texts = process_audio_files(val_audio_files, classifier, tokenizer, dir)

            # # Рассчитываем WER для валидационного набора
            # wer = jiwer.wer(val_reference_texts, val_hypothesis_texts)
            print(f"Эпоха {epoch + 1}")

##### Использование модели для транскрибации и классификации текста

In [6]:
# директория, где лежат файлы, которые мы хотим обработать
result_dir = '../data/train/hr_bot_synt'

result_files = os.listdir(result_dir)

for i in range(len(result_files)):
    result_files[i] = {"file": result_files[i], "id": result_files[i][:-4]}

# Обработка файлов и получение результатов
transcription_results = process_audio_files(result_files, classifier, tokenizer, result_dir)

# Сохранение результатов в JSON
with open("transcriptions.json", "w", encoding="utf-8") as f:
    json.dump(transcription_results, f, ensure_ascii=False, indent=4)

NameError: name 'process_audio_files' is not defined

Вывод результатов в консоль

In [ ]:
# выводим результат
# print(json.dumps(transcription_results, indent=4, ensure_ascii=False))